# Homework 7
Jiannan Chen, Xiaoyi Duan, Xukun Liu

In this homework, we try to draw some conlusions based on our analysis and visualizations. Our visualizations include a geographical scatter plot, a geographical heatmap, and a text-processing heatmap. 

This project is very different from our midterm project because we analyze and visualize English texts here, whereas we processed text dataset in Chinese for our midterm project. English is very different from Chinese in terms of construction and grammar, hence we used different methods to do the analysis. And we were dealing with well-formed text such as sentences in novels such as 'Ulysses' and 'Tale of Two Cities', but we may process ill-formed text from blogs in our this project. 

To make it more convenient for you to check, we put all of the visualization part in the upper area of this page followed by all the functions and code.

The 'Results dataset.csv' can be downloaded from this [link](#https://690v.blob.core.windows.net/690v/Results%20dataset.csv). 

**To run the code, you may need to download nltk package as well as the stop words dictionary. Or you can load the text_new data and the len_list data to skip the tokenize step.* 

In [99]:
# -*- coding: utf-8 -*-
from bokeh.core.properties import field
from ipywidgets import interact
from bokeh.io import push_notebook, output_notebook
from bokeh.layouts import layout
from bokeh.models import (
    ColumnDataSource, HoverTool, SingleIntervalTicker, Slider, Button, Label,
    CategoricalColorMapper, CustomJS,
)
from bokeh.models import LogColorMapper, LinearColorMapper, BasicTicker, PrintfTickFormatter, ColorBar
from bokeh.models import HoverTool, LassoSelectTool, WheelPanTool, WheelZoomTool, BoxZoomTool, \
                         ResetTool, UndoTool, RedoTool, SaveTool, BoxSelectTool, \
                         PanTool, PolySelectTool, CustomJS, Select
from bokeh.palettes import Spectral6, brewer
from bokeh.palettes import RdGy, PuBu
from bokeh.plotting import figure, show
from bokeh.layouts import column, widgetbox, row, gridplot, layout
from bokeh.charts import HeatMap
from bkcharts import Bar, output_file, show
from bokeh.models.widgets import Div
from bokeh.models.widgets import Dropdown
import warnings

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

import csv
import numpy as np
import pandas as pd
from collections import Counter
from collections import defaultdict
from datetime import datetime
from math import pi
from operator import itemgetter

from nltk.tokenize import TweetTokenizer
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
import string
from nltk.corpus import stopwords

output_notebook()

Loading BokehJS ...

# Geographical Visualization
With our clustering result on twitter post in which we clustered the posts into two categories (relative to the epidemic/irrelative to the epidemic), we plotted the posts source relative to the epidemic on the geographical map. With the interaction of changing date below, we could see how the wind direction changed each day and how the posts sources changed with the wind direction. First of all, we could see that most of the posts sources are in the downtown. This is strange because Eastside had the largest population but it has a very low posts density. We have two possible explaination for this phenomenon. One is that the epidemic was popular in downtown and did not spread widely to other districts. Another is that most people went to work in the downtown and they posts tweets mostly in the daytime. 

We could barely see any shifts of the core of the tweets sources, however we could see the number of sources increased in these districts to the direction of the wind. This may indicate that the epidemic could spread by air. 

Based on our analysis above, we would provide the government with following suggestions: 1) distribute medical resources in the downtown and in districts to the direction of the wind. 2) suggest citizens to take measures to avoid airborne infection such as wearing breathing masks and spending less time outdoors.

In [112]:
show(plot,notebook_handle=True)

In [73]:
from bokeh.io import push_notebook
import ipywidgets as widgets
from IPython.display import display
import datetime

nextdaybutton = widgets.Button(description="5/1")
previousdaybutton = widgets.Button(description="4/29")
currentdate=datetime.date(2011,4,30)
display(previousdaybutton)
display(nextdaybutton)

#data part

def nextdaygo(button):
    global currentdate
    previousdate=currentdate
    currentdate=currentdate+datetime.timedelta(days=1)
    nextdate=currentdate+datetime.timedelta(days=1)
    previousdaybutton.description=str(previousdate.month)+'/'+str(previousdate.day)
    nextdaybutton.description=str(nextdate.month)+'/'+str(nextdate.day)
    updatemapdata(currentdate)
    
def previousdaygo(button):
    global currentdate
    nextdate=currentdate
    currentdate=currentdate-datetime.timedelta(days=1)
    previousdate=currentdate-datetime.timedelta(days=1)
    previousdaybutton.description=str(previousdate.month)+'/'+str(previousdate.day)
    nextdaybutton.description=str(nextdate.month)+'/'+str(nextdate.day)
    updatemapdata(currentdate)
    
def updatemapdata(date):
    #date is the date to extract data to show on map
    currentdatedata=data_new[(data_new.created_month==date.month) & (data_new.created_day==date.day)]
    twitterhandle.data_source.data=dict(defaultx=currentdatedata['defaultx'],defaulty=currentdatedata['defaulty'])
    windhandle.data_source.data=dict(url = [arrowurl],size=[currentdatedata['windsize'].iloc[0]],angle=[currentdatedata['wind_angle'].iloc[0]])
    plot.title.text="Twitter posts on "+ str(date.month)+'/'+str(date.day)+' '+currentdatedata.Weather.iloc[0]
    push_notebook()
    
nextdaybutton.on_click(nextdaygo)
previousdaybutton.on_click(previousdaygo)

# Geographical Heatmap
As we can see from the chart below, we show the scatterplots based on twitter dates and locations. You could see the trending of scatterplots by modify the code above. The screenshots we provide here are from May 10, 17, 18, 19, 20. You could see a huge difference between May 17 and May 18 in here. Before May.18, it is easy to observe that less people were talking about their health and many of them only felt sick or wanted spill, but not fever or flu. However, things become extremly strange after May 17. The images from May 18 to 20 could show the transition from fever to chills obviously (The red dots to blue dots). We could also see a huge increase of number of twitters after May 17, and the locations of twitters are concentrated to downtown at most of time. Based on the observation, a clear epidemic is observed.

<img src="Vastopolis_Map.png", width="833", height="424">

<img src="5_11.png", alt="May 11", width=800>
<img src="5_17.png", alt="May 17", width=800>
<img src="5_18.png", alt="May 18", width=800>
<img src="5_19.png", alt="May 19", width=800>
<img src="5_20.png", alt="May 20", width=800>

In [113]:
show(p_loc)

# Data processing and the Heatmap
In order to see the difference of words' frequencies among different time intervals, we need to ignore the wrong data information in the date column from csv file such as - ["-Fire", "Fire-", "up", "One", "Radio", "Three", "SC", "...", "", ":\*", "\<3)", "."]. After skipping the stop words (in, and, the) and stemming our sentences, we calculate the total frequency of each word from the twitter file. Then, select the words which may be related to epidemic by human dection. The heatmaps will show the words we select clearly. Here is the explanation of three types of heatmap: 

From the heatmap one, we could see that a potential epidemic may be happening. It is very easy to find out that words like "chills", "fever", "flu", "cold", terrible", "headache", "fatigue", "diarrhea", "sweats" and "cough" are discussed much more after 17th, May. It indicates some bad things are happening in the town.
From the heatmap two (please use drop down below), we could see that people tends to talk about more through twitter in the afternoon and night. It is an interesting discovery but it did not relate to the epidemic issue.
By using the "Weather.csv", we could see the heatmap three. It is easy to find out word like "chills" or "flu" are discussed more in the cloudy day. We could make an assumption that weather is a potential influential factor for increasing epidemic. This idea could also be helpful for clustering.

In [110]:
show(column(p_heat1), notebook_handle = True)

In [111]:
interact(update, heatmap_type=["dates", "datetime", "weather"])

<function __main__.update>

In [75]:
import pandas as pd
data=pd.read_csv('Results dataset.csv',encoding='latin-1')

In [48]:
## Read data from microblog dataset
rowlist = []

with open('Results dataset.csv', 'r', encoding = 'latin-1') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader, None)
    for row in reader:
        rowlist.append(row)

In [15]:
text_list = []
for r in rowlist:
    text_list.append(r[3])

In [7]:
keywords = ["bad", "sick", 
        "fever", "flu", 
         "terrible", "headache", "dead", "breath", "stomach", "pneumonia", "fatigue", "blood", "diarrhea",  
        "sweats", "cough", "fire", "bad", "oh", "god", "accident", "truck", "trucks", "omg", "hurts", "smoke"]

In [6]:
# print(data)
def int2str(l):
    res = []
    for i in l:
        res.append(str(i))
    return res

In [12]:
### Twitter tokenize

stop = stopwords.words('english') + list(string.punctuation)

def nltk_tokenizer(doc):
    st = SnowballStemmer("english")
    tknzr = TweetTokenizer()
    tokens = tknzr.tokenize(doc)
    lowered_tokens = map(lambda t: t.lower(), tokens)
    result_list = []
    for token in lowered_tokens:
        if token in stop:
            continue
        stemmedToken = st.stem(token)
        result_list.append(token)
    return result_list

In [8]:
## build vectorizer

vectorizer = CountVectorizer()
analyze = vectorizer.fit(keywords)

# Build column data for twitter length and keywords count
text_new = []
len_list = []

for text in text_list:
    tokens = nltk_tokenizer(text)
    len_list.append(len(tokens))
    sentence = ' '.join(tokens)
    text_new.append(sentence)  
    
## serialize the text_new in disk
# import json
# with open('tokenized_text.txt', 'w') as outfile:
#     json.dump(text_new, outfile)
# outfile.close()

## serialize the len_list in disk
# import json
# with open('len_list.txt', 'w') as outfile:
#     json.dump(len_list, outfile)
# outfile.close()

In [9]:
### read text_new from disk
# import json

# text_new = []
# with open('tokenized_text.txt') as json_data:
#     text_new = json.load(json_data)

In [10]:
### read text_new from disk
# import json

# len_list = []
# with open('len_list.txt') as json_data:
#     len_list = json.load(json_data)

In [65]:
#build text_vectors using vectorizer
text_vector = vectorizer.transform(text_new).toarray()

In [12]:
## appending the length column and the keywords count column to the numpy array
arr_len = np.asarray(len_list)
arr_len = arr_len.reshape((arr_len.shape[0], 1))
arr_sum = np.sum(text_vector, axis = 1)
arr_sum = arr_sum.reshape((arr_sum.shape[0], 1))
array = np.hstack((arr_len, arr_sum, text_vector))
array = array.astype(float)

In [11]:
## serialize the array to disk
# np.save('array.npy', array)

In [14]:
### constructing the dataframe
import pandas as pd
columns = ['length', 'positive_words']
idx = [i for i in range(24)]
columns += idx
arry_df = pd.DataFrame(data = array, columns = columns)

In [16]:
## sample the data so that we can visualize it within the notebook
# arry_df = arry_df.sample(frac = 0.01, random_state = 0, axis = 0)

In [15]:
### the trainset for clustering
trainset = arry_df.values[:, 2:]

In [68]:
### perform Kmeans clsutering 


cate = list(arry_df.columns)
index = list(arry_df.index)

xaxis_name = cate[1]
yaxis_name = cate[0]

cluster = KMeans(n_clusters=2, random_state=0, init='k-means++').fit(trainset)
cluster_labels = int2str(list(cluster.labels_))

In [69]:
# print(cluster_labels)
k0 = 0
k1 = 0
for c in cluster_labels:
    if c == '0':
        k0 += 1
    elif c == '1':
        k1 += 1

print(k0, k1)  

973263 49811


In [56]:
counter = 0
rows = []
labels = []
for c in cluster_labels:
    if c == '0':
        pass
    else:
        rows.append(counter)
        labels.append(c)
    counter += 1

In [58]:
new_df = arry_df.loc[rows, :]

In [76]:
data_new = data.loc[rows, :]
data_new = data_new.reset_index()

In [77]:
import numpy as np
from bokeh.models import ColumnDataSource,Range1d,Plot,Rect
from bokeh.models.glyphs import ImageURL
from bokeh.io import output_notebook, show
import math

mapurl="map.png"
arrowurl="arrow.png"

sizefactor=1.6
imagewidth=521 #ori img size yue = 5210*2650
imageheight=265
actualwidth=833#int(sizefactor*imagewidth)
actualheight=424#int(sizefactor*imageheight)
dotsize=int(sizefactor*4)

#get initial data on 4/30
initialdata430=data_new[(data_new.created_month==4) & (data_new.created_day==30)]

imagesource = ColumnDataSource(dict(url = [mapurl]))
windsource=ColumnDataSource(dict(url = [arrowurl],size=[initialdata430['windsize'].iloc[0]],angle=[initialdata430['wind_angle'].iloc[0]]))

plot = Plot(x_range=Range1d(0,actualwidth), y_range=Range1d(0,actualheight), plot_width=actualwidth, plot_height=actualheight,min_border=0)
plot.title.text="Twitter posts on 4/30 cloudy"
plot.title.align='center'

mapimg = ImageURL(url="url", x=0, y=0, w=actualwidth, h=actualheight, anchor="bottom_left")
windimg=ImageURL(url="url", x=833/2, y=100,w="size",h="size",anchor="bottom_left",angle="angle",global_alpha=0.4)

plot.add_glyph(imagesource, mapimg)
windhandle=plot.add_glyph(windsource, windimg)


datasource = ColumnDataSource(data=initialdata430)
twittersquares = Rect(x="defaultx", y="defaulty", width=dotsize, height=dotsize, angle=0, fill_color="red",line_color="red")
twitterhandle=plot.add_glyph(datasource, twittersquares)

output_notebook()


Loading BokehJS ...

In [70]:
from bokeh.io import push_notebook
import ipywidgets as widgets
from IPython.display import display
import datetime

nextdaybutton = widgets.Button(description="5/1")
previousdaybutton = widgets.Button(description="4/29")
currentdate=datetime.date(2011,4,30)
display(previousdaybutton)
display(nextdaybutton)

#data part

def nextdaygo(button):
    global currentdate
    previousdate=currentdate
    currentdate=currentdate+datetime.timedelta(days=1)
    nextdate=currentdate+datetime.timedelta(days=1)
    previousdaybutton.description=str(previousdate.month)+'/'+str(previousdate.day)
    nextdaybutton.description=str(nextdate.month)+'/'+str(nextdate.day)
    updatemapdata(currentdate)
    
def previousdaygo(button):
    global currentdate
    nextdate=currentdate
    currentdate=currentdate-datetime.timedelta(days=1)
    previousdate=currentdate-datetime.timedelta(days=1)
    previousdaybutton.description=str(previousdate.month)+'/'+str(previousdate.day)
    nextdaybutton.description=str(nextdate.month)+'/'+str(nextdate.day)
    updatemapdata(currentdate)
    
def updatemapdata(date):
    #date is the date to extract data to show on map
    currentdatedata=data_new[(data_new.created_month==date.month) & (data_new.created_day==date.day)]
    twitterhandle.data_source.data=dict(defaultx=currentdatedata['defaultx'],defaulty=currentdatedata['defaulty'])
    windhandle.data_source.data=dict(url = [arrowurl],size=[currentdatedata['windsize'].iloc[0]],angle=[currentdatedata['wind_angle'].iloc[0]])
    plot.title.text="Twitter posts on "+ str(date.month)+'/'+str(date.day)+' '+currentdatedata.Weather.iloc[0]
    push_notebook()
    
nextdaybutton.on_click(nextdaygo)
previousdaybutton.on_click(previousdaygo)

In [82]:
stop = stopwords.words('english') + list(string.punctuation)

def nltk_tokenizer(doc):
    st = SnowballStemmer("english")
    tknzr = TweetTokenizer()
    tokens = tknzr.tokenize(doc)
    lowered_tokens = map(lambda t: t.lower(), tokens)
    result_list = []
    for token in lowered_tokens:
        if token in stop:
            continue
        stemmedToken = st.stem(token)
        result_list.append(token)
    return result_list

def top_n(d, n):
    return sorted(d.items(), key=itemgetter(1), reverse=True)[:n]

In [84]:
id_list = []
time = []
loc = []
text = []

# alternative method: bigram

with open("Microblogs.csv", "r", encoding='latin-1') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')

    for row in reader:
        id_list.append(row[0])
        time.append(row[1])
        loc.append(row[2])
        text.append(row[3])

del id_list[0]; del time[0]; del loc[0]; del text[0]

# bow = Counter()

for i in range(len(text)):
    # print i
    text[i] = nltk_tokenizer(text[i])
#     for token in text[i]:
#         bow[token] += 1

# print top_n(bow, 500)

In [85]:
target_words = ["chills", "sick", "fever", "flu", "hot", "cold", "headache", 
                "spill", "sweats", "cough"]
target_colors = ["#0000e6", "#2eb82e", "#ff6600", "#9900ff", "#ffcc00", "#00ccff", "#ff4dff",
                "#00e600", "#bf4040", "#c68c53"]

for i in range(len(loc)):
    loc[i] = loc[i].split()
    loc[i][0] = float(loc[i][0])
    loc[i][1] = float(loc[i][1])

In [86]:
loc = np.array(loc)
min_col = loc.min(axis=0)
for i in range(len(loc)):
    loc[i][0] -= min_col[0]
    loc[i][1] -= min_col[1]

In [87]:
scatter_map = defaultdict(defaultdict)
order_date = ['4/30/2011', '5/1/2011', '5/2/2011', '5/3/2011', '5/4/2011', '5/5/2011', '5/6/2011', '5/7/2011', '5/8/2011', 
              '5/9/2011', '5/10/2011', '5/11/2011', '5/12/2011', '5/13/2011', '5/14/2011', '5/15/2011', '5/16/2011', '5/17/2011', 
              '5/18/2011', '5/19/2011', '5/20/2011']
for d in order_date:
    for w in target_words:
        scatter_map[d][w] = []

In [88]:
time_day = [] # 18/05/2011
# time_datetime = [] # 13:25
for s in time:
    tmp = s.split()
    time_day.append(tmp[0])
    # time_datetime.append(tmp[1])

In [89]:
loc = loc.tolist()
for i, tt in enumerate(text):
    for token in tt:
        if token in target_words:
            scatter_map[time_day[i]][token].append(loc[i])

In [91]:
"""Figure 1: location heatmap by using scatter plot"""

p_loc = figure(plot_width=833, plot_height=424,
            tools=[BoxZoomTool(), LassoSelectTool(), WheelPanTool(),
                   WheelZoomTool(), ResetTool(), UndoTool(), RedoTool(),
                   SaveTool(), BoxSelectTool(), PanTool(), PolySelectTool()],
            title='Location heatmap (changed by date)')

select_date = "5/20/2011"
for i in range(len(scatter_map[select_date])):
    data_tmp = np.array(scatter_map[select_date][target_words[i]])
    xx = data_tmp[:, 0]
    yy = data_tmp[:, 1]
    p_loc.circle(xx, yy, size=10, color=target_colors[i], legend=target_words[i], alpha = 0.1)

p_loc.xaxis.axis_label = 'location_x'
p_loc.yaxis.axis_label = 'location_y'
p_loc.legend.click_policy="hide"


In [92]:
show(p_loc)

In [96]:
special_words = ["fire", "life", "better", "feel", "hope", "bad", "great", "chills", "sleep", "feeling", "sick", "crazy", 
        "fever", "flu", "annoying", "hot", "cold", "cool", "worse", "health", "hurricane", 
         "crisis", "terrible", "headache", "dead", "spill", "breath", "episode", "stomach", "pneumonia",
        "sucks", "fatigue", "trending", "blood", "diarrhea", "problem", "rain", "boring", "weather", 
        "sweats", "weird", "cough", "worst", "bored", "sunday", "wtf","mad", "smoke"]
# 
len(special_words)

48

In [97]:
"""Heatmap 1: date vs. words"""
heatmap_time = defaultdict(Counter)

time_day = [] # 18/05/2011
time_datetime = [] # 13:25
for s in time:
    tmp = s.split()
    time_day.append(tmp[0])
    time_datetime.append(tmp[1])

for i in range(len(text)):
    # print i
    for token in text[i]:
        if token in special_words:
            heatmap_time[time_day[i]][token] += 1

# print heatmap_time

In [100]:
"""Heatmap 2: hour vs. words"""
order_time_interval = ["00:00-04:00", "04:00-08:00", "08:00-12:00", "12:00-16:00", "16:00-20:00", "20:00-23:59"]
time_interval = [datetime(2017, 8, 8, 4, 0), datetime(2017, 8, 8, 8, 0), 
                 datetime(2017, 8, 8, 12, 0), datetime(2017, 8, 8, 16, 0), datetime(2017, 8, 8, 20, 0),
                 datetime(2017, 8, 8, 23, 59)]
heatmap_hour = defaultdict(Counter)

year_default = 2017; month_default = 8; day_default = 8

# time_datetime[31296] "-Fire" processing missing values
syntax_errors = ["-Fire", "Fire-", "up", "One", "Radio", "Three", "SC", "...", "", ':*', '&lt;3)', '.']
# We also try heatmap several times, delete the meaningless words like sunday (before the visualization, we think it may related)

for i, tt in enumerate(time_datetime):
    # print i
    if tt in syntax_errors:
        continue
    tmp = tt.split(":")
    hour = int(tmp[0])
    minutes = int(tmp[1])
    if datetime(year_default, month_default, day_default, hour, minutes) < time_interval[0]:
        for token in text[i]:
            heatmap_hour["00:00-4:00"][token] += 1
    elif datetime(year_default, month_default, day_default, hour, minutes) < time_interval[1]:
        for token in text[i]:
            heatmap_hour["04:00-8:00"][token] += 1
    elif datetime(year_default, month_default, day_default, hour, minutes) < time_interval[2]:
        for token in text[i]:
            heatmap_hour["08:00-12:00"][token] += 1
    elif datetime(year_default, month_default, day_default, hour, minutes) < time_interval[3]:
        for token in text[i]:
            heatmap_hour["12:00-16:00"][token] += 1
    elif datetime(year_default, month_default, day_default, hour, minutes) < time_interval[4]:
        for token in text[i]:
            heatmap_hour["16:00-20:00"][token] += 1
    elif datetime(year_default, month_default, day_default, hour, minutes) <= time_interval[5]:
        for token in text[i]:
            heatmap_hour["20:00-23:59"][token] += 1


In [101]:
heat2_X = []; heat2_Y = []; heat2_freq_stack = []
for i in special_words:
    for j in order_time_interval:
        heat2_X.append(i)
        heat2_Y.append(j)
        heat2_freq_stack.append(heatmap_hour[j][i])

In [102]:
order_date = ['4/30/2011', '5/1/2011', '5/2/2011', '5/3/2011', '5/4/2011', '5/5/2011', '5/6/2011', '5/7/2011', '5/8/2011', 
              '5/9/2011', '5/10/2011', '5/11/2011', '5/12/2011', '5/13/2011', '5/14/2011', '5/15/2011', '5/16/2011', '5/17/2011', 
              '5/18/2011', '5/19/2011', '5/20/2011']

In [104]:
""" Heatmap 3: weather vs. words """

weather_date_map = dict()
with open("Weather.csv", "r", encoding = 'latin-1') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        weather_date_map[row[0]] = row[1]
# print weather_date_map

heatmap_weather = defaultdict(Counter)
for k in heatmap_time.keys():
    for token in heatmap_time[k]:
        heatmap_weather[weather_date_map[k]][token] += heatmap_time[k][token]

In [105]:
order_weather_states = list(heatmap_weather.keys())
print(order_weather_states)
heat3_X = []; heat3_Y = []; heat3_freq_stack = []
for i in special_words:
    for j in order_weather_states:
        heat3_X.append(i)
        heat3_Y.append(j)
        heat3_freq_stack.append(heatmap_weather[j][i])

['clear', 'showers', 'rain', 'cloudy']


In [106]:
colors = ["#330000", "#4d0000", "#660000", "#800000", "#990000", "#b30000", "#cc0000", "#e60000", "#ff0000", 
         "#ff1a1a", "#ff3333", "#ff4d4d", "#ff6666", "#ff8080", "#ff9999", "#ffb3b3", "#ffcccc", 
         "#ffe6e6", "#ffffff"]
# colors = ["#003399", "#3377ff", "#809fff", "#e6eeff", "#ffffff", "#ffe6e6", "#ff6666", "#ff1a1a", "#b30000"]
colors = colors[::-1]
hover_heat = HoverTool(tooltips=[
    ("token freq", "@freq")
])

p_heat1 = figure(title="Heatmap - Date VS. Words", plot_width=1100, plot_height=500,
                x_range=special_words, y_range=order_date, x_axis_location="above",
#            x_range=special_words, y_range=order_date,
#            x_axis_location="above", plot_width=1000, plot_height=500)
          tools=[hover_heat, BoxZoomTool(), LassoSelectTool(), WheelPanTool(),
                   WheelZoomTool(), ResetTool(), UndoTool(), RedoTool(),
                    SaveTool(), BoxSelectTool(), PanTool(), PolySelectTool()], 
            toolbar_location='below')

p_heat1.axis.major_label_text_font_size = "6pt"
p_heat1.xaxis.major_label_orientation = pi / 3
p_heat1.axis.major_label_standoff = 0
p_heat1.axis.axis_line_color = None
p_heat1.axis.major_tick_line_color = None

heat1_X = []; heat1_Y = []; heat1_freq_stack = []
for i in special_words:
    for j in order_date:
        heat1_X.append(i)
        heat1_Y.append(j)
        heat1_freq_stack.append(heatmap_time[j][i])

# print min(heat1_freq_stack), max(heat1_freq_stack)
mapper_heat = LogColorMapper(palette=colors, low=min(heat1_freq_stack), high=max(heat1_freq_stack))

source_heat = ColumnDataSource(
    data=dict(
        x_loc=heat1_X,
        y_loc=heat1_Y,
        freq=heat1_freq_stack
    )
)

rec_plot = p_heat1.rect(x='x_loc', y='y_loc', width=1, height=1, source=source_heat,
       fill_color={'field':'freq', 'transform':mapper_heat},
       line_color="white", alpha=0.7)

# "{'field':'freq', 'transform':mapper_heat}"

# p_heat1.select_one(HoverTool).tooltips = [
#      ('token freq', '@freq'),
# ]

In [107]:
def update(heatmap_type):
    if heatmap_type == "dates":
        p_heat1.y_range.factors = order_date
        rec_plot.data_source.data["x_loc"] = heat1_X
        rec_plot.data_source.data["y_loc"] = heat1_Y
        rec_plot.data_source.data["freq"] = heat1_freq_stack
        mapper_heat = LogColorMapper(palette=colors, low=min(heat1_freq_stack), high=max(heat1_freq_stack))
        rec_plot.glyph.fill_color['field'] = 'freq'
        rec_plot.glyph.fill_color['transform'] = mapper_heat
        p_heat1.title.text = 'Heatmap - Date VS. Words'
    elif heatmap_type == "datetime":
        p_heat1.y_range.factors = order_time_interval
        rec_plot.data_source.data["x_loc"] = heat2_X
        rec_plot.data_source.data["y_loc"] = heat2_Y
        rec_plot.data_source.data["freq"] = heat2_freq_stack
        mapper_heat = LogColorMapper(palette=colors, low=min(heat2_freq_stack), high=max(heat2_freq_stack))
        rec_plot.glyph.fill_color['field'] = 'freq'
        rec_plot.glyph.fill_color['transform'] = mapper_heat
        p_heat1.title.text = 'Heatmap - Datetime VS. Words'
    elif heatmap_type == "weather":
        p_heat1.y_range.factors = order_weather_states
        rec_plot.data_source.data["x_loc"] = heat3_X
        rec_plot.data_source.data["y_loc"] = heat3_Y
        rec_plot.data_source.data["freq"] = heat3_freq_stack
        mapper_heat = LogColorMapper(palette=colors, low=min(heat3_freq_stack), high=max(heat3_freq_stack))
        rec_plot.glyph.fill_color['field'] = 'freq'
        rec_plot.glyph.fill_color['transform'] = mapper_heat
        p_heat1.title.text = 'Heatmap - Wheather VS. Words'
        
    push_notebook()

In [108]:
show(column(p_heat1), notebook_handle = True)

In [109]:
interact(update, heatmap_type=["dates", "datetime", "weather"])

<function __main__.update>